# Assignment Two:  Sentiment Classification

For this exercise you will be using the "SemEval 2017 task 4" corpus provided on the module website, available through the following link: https://warwick.ac.uk/fac/sci/dcs/teaching/material/cs918/semeval-tweets.tar.bz2 You will focus particularly on Subtask A, i.e. classifying the overall sentiment of a tweet as positive, negative or neutral.

You are requested to produce a *Jupyter notebook* for the coursework submission. The input to your program is the SemEval data downloaded. Note that TAs need to run your program on their own machine by using the original SemEval data. As such, don’t submit a Python program that takes as input some preprocessed files.

#### Import necessary packages
You may import more packages here.

In [2]:
# Import necessary packages here
import re
from os.path import join
import numpy as np

In [3]:
# Define test sets
dataDir = '../semeval-tweets'
testsetStrings = ['twitter-test1.txt', 'twitter-test2.txt', 'twitter-test3.txt']
testsets = [join(dataDir, t) for t in testsetStrings]
print(testsets)

['../semeval-tweets/twitter-test1.txt', '../semeval-tweets/twitter-test2.txt', '../semeval-tweets/twitter-test3.txt']


In [7]:
# checking the structure of the dataset
with open(testsets[0], 'r', encoding='utf8') as f1:
    i = 0
    for line in f1:
        fields = line.split('\t')
        if i < 30:
            length = len(fields[2])
            if length > 130:
                print(f"Lenght: {length}")
                print(fields[0])  # 1st column - tweet ID
                print(fields[1])  # 2nd column - tweet sentiment
                print(fields[2])  # 3rd column - tweet text
                i += 1

# preprocessing questions and notes: 
  # -> what about removing the @usernames, is it advisable?
  # -> need to remove URLs!
  # -> There is a lot of noise/mistakes in the data and absence of interpunction.
  # -> what about adding of the starting token?  

Lenght: 139
102313285628711403
neutral
"Bing one-ups knowledge graph, hires Encyclopaedia Britannica to supply results:   It may have retired from the cut-throat world of pr..."

Lenght: 139
653274888624828198
neutral
"On Thursday, concealed-carry gun license holders will be given a new right in the state of Oklahoma: the ability... http://t.co/oSgGHKi1"

Lenght: 137
420747042670198316
negative
Miyagi just got banned from yoga. He was caught him sniffing the sphincter of the girl in front of him. There may be police involvement!

Lenght: 135
822064800445716046
neutral
Join us tonight at Boston Pizza - Centre on Barton for THURSDAY NIGHT FOOTBALL! Tonight the Chiefs take on the... http://t.co/iegTxPQv

Lenght: 139
055480020953212084
neutral
"#FX NEW YORK, Oct 18 (Reuters) - The Federal Reserve provided $4.701 billion of liquidity to the ... http://t.co/BJhIQTtO #EUR #AUD #CAD"

Lenght: 132
429443270273347255
neutral
"13 April 1996, History is made, as the MetroStars and the Los Angeles 

In [13]:
# Skeleton: Evaluation code for the test sets
def read_test(testset):
    '''
    reading the testset and return a dictionary with: ID -> sentiment
    :param testset: str, the file name of the testset to compare
    '''
    id_gts = {}  # init the dictionary
    with open(testset, 'r', encoding='utf8') as fh:
        for line in fh:
            fields = line.split('\t')
            tweetid = fields[0]
            gt = fields[1]
            id_gts[tweetid] = gt

    return id_gts


def confusion(id_preds, testset, classifier):
    '''
    print the confusion matrix of {'positive', 'netative'} between preds and testset
    :param id_preds: a dictionary of predictions formated as {<tweetid>:<sentiment>, ... }
    :param testset: str, the file name of the testset to compare
    :classifier: str, the name of the classifier
    '''
    id_gts = read_test(testset)

    gts = []
    for m, c1 in id_gts.items():
        if c1 not in gts:
            gts.append(c1)

    gts = ['positive', 'negative', 'neutral']

    conf = {}
    for c1 in gts:
        conf[c1] = {}
        for c2 in gts:
            conf[c1][c2] = 0

    for tweetid, gt in id_gts.items():
        if tweetid in id_preds:
            pred = id_preds[tweetid]
        else:
            pred = 'neutral'
        conf[pred][gt] += 1

    print(''.ljust(12) + '  '.join(gts))

    for c1 in gts:
        print(c1.ljust(12), end='')
        for c2 in gts:
            if sum(conf[c1].values()) > 0:
                print('%.3f     ' % (conf[c1][c2] / float(sum(conf[c1].values()))), end='')
            else:
                print('0.000     ', end='')
        print('')

    print('')


def evaluate(id_preds, testset, classifier):
    '''
    print the macro-F1 score of {'positive', 'netative'} between preds and testset
    :param id_preds: a dictionary of predictions formated as {<tweetid>:<sentiment>, ... }
    :param testset: str, the file name of the testset to compare
    :classifier: str, the name of the classifier
    '''
    id_gts = read_test(testset)

    acc_by_class = {}
    for gt in ['positive', 'negative', 'neutral']:
        acc_by_class[gt] = {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0}

    catf1s = {}

    ok = 0
    for tweetid, gt in id_gts.items():
        if tweetid in id_preds:
            pred = id_preds[tweetid]
        else:
            pred = 'neutral'

        if gt == pred:
            ok += 1
            acc_by_class[gt]['tp'] += 1
        else:
            acc_by_class[gt]['fn'] += 1
            acc_by_class[pred]['fp'] += 1

    catcount = 0
    itemcount = 0
    macro = {'p': 0, 'r': 0, 'f1': 0}
    micro = {'p': 0, 'r': 0, 'f1': 0}
    semevalmacro = {'p': 0, 'r': 0, 'f1': 0}

    microtp = 0
    microfp = 0
    microtn = 0
    microfn = 0
    for cat, acc in acc_by_class.items():
        catcount += 1

        microtp += acc['tp']
        microfp += acc['fp']
        microtn += acc['tn']
        microfn += acc['fn']

        p = 0
        if (acc['tp'] + acc['fp']) > 0:
            p = float(acc['tp']) / (acc['tp'] + acc['fp'])

        r = 0
        if (acc['tp'] + acc['fn']) > 0:
            r = float(acc['tp']) / (acc['tp'] + acc['fn'])

        f1 = 0
        if (p + r) > 0:
            f1 = 2 * p * r / (p + r)

        catf1s[cat] = f1

        n = acc['tp'] + acc['fn']

        macro['p'] += p
        macro['r'] += r
        macro['f1'] += f1

        if cat in ['positive', 'negative']:
            semevalmacro['p'] += p
            semevalmacro['r'] += r
            semevalmacro['f1'] += f1

        itemcount += n

    micro['p'] = float(microtp) / float(microtp + microfp)
    micro['r'] = float(microtp) / float(microtp + microfn)
    micro['f1'] = 2 * float(micro['p']) * micro['r'] / float(micro['p'] + micro['r'])

    semevalmacrof1 = semevalmacro['f1'] / 2

    print(testset + ' (' + classifier + '): %.3f' % semevalmacrof1)

In [14]:
# testing the evaluation functions
tweetDict = read_test('../semeval-tweets/twitter-test1.txt')
confusion(tweetDict, '../semeval-tweets/twitter-test1.txt', "PerfectClassifier")
evaluate(tweetDict, '../semeval-tweets/twitter-test1.txt', "PerfectClassifier")

            positive  negative  neutral
positive    1.000     0.000     0.000     
negative    0.000     1.000     0.000     
neutral     0.000     0.000     1.000     

../semeval-tweets/twitter-test1.txt (PerfectClassifier): 1.000


---
### Load training set, dev set and testing set
Here, you need to load the training set, the development set and the test set. For better classification results, you may need to preprocess tweets before sending them to the classifiers.

In [40]:
# Load training set, dev set and testing set

dataDir = '../semeval-tweets'  # change to the proper directory
datasetStrings = ['twitter-training-data.txt', 'twitter-test1.txt', 'twitter-test2.txt', 'twitter-test3.txt', 'twitter-dev-data.txt']
datasets = [join(dataDir, t) for t in datasetStrings]

tweet_IDs = {}          # init dictionary with tweet IDs
tweet_sentiments = {}   # init dictionary with sentiments
tweet_texts = {}        # init dictionary with tweet texts

for DatasetString in datasets:
    data_ID, data_sent, data_text  = {}, {}, {}    # temp dictionaries
    with open(DatasetString, 'r', encoding='utf8') as f1:
        for i, line in enumerate(f1):
            fields = line.split('\t')
            data_ID[i] = fields[0]            # tweet IDs
            data_sent[fields[0]] = fields[1]  # sentiments
            data_text[fields[0]] = fields[2]  # tweet text
    tweet_IDs[DatasetString] = data_ID
    tweet_sentiments[DatasetString] = data_sent
    tweet_texts[DatasetString] = data_text

# sentiment dictionaries
sent_train = tweet_sentiments[datasets[0]]
sent_test1 = tweet_sentiments[datasets[1]]
sent_test2 = tweet_sentiments[datasets[2]]
sent_test3 = tweet_sentiments[datasets[3]]
sent_dev = tweet_sentiments[datasets[4]]

# tweet text dictionaries
text_train = tweet_texts[datasets[0]]
text_test1 = tweet_texts[datasets[1]]
text_test2 = tweet_texts[datasets[2]]
text_test3 = tweet_texts[datasets[3]]
text_dev = tweet_texts[datasets[4]]

# tweet IDs dictionaries
IDs_train = tweet_IDs[datasets[0]]
IDs_test1 = tweet_IDs[datasets[1]]
IDs_test2 = tweet_IDs[datasets[2]]
IDs_test3 = tweet_IDs[datasets[3]]
IDs_dev = tweet_IDs[datasets[4]]


## examples and tests
# id = IDs_train[0]
# id_dev = IDs_dev[0]
# id1 = IDs_test1[0]
# id2 = IDs_test2[0]
# id3 = IDs_test3[0]
# print(f"-ID:{id} \n-TEXT:{text_train[id]}-SENTIMENT: {sent_train[id]}\n")
# print(f"-ID:{id_dev} \n-TEXT:{text_dev[id_dev]}-SENTIMENT: {sent_dev[id_dev]}\n")
# print(f"-ID:{id1} \n-TEXT:{text_test1[id1]}-SENTIMENT: {sent_test1[id1]}\n")
# print(f"-ID:{id2} \n-TEXT:{text_test2[id2]}-SENTIMENT: {sent_test2[id2]}\n")
# print(f"-ID:{id3} \n-TEXT:{text_test3[id3]}-SENTIMENT: {sent_test3[id3]}\n")
# print(len(IDs_train.keys()), len(text_train.keys()), len(sent_train.keys()))  # 45101
# print(len(IDs_test1.keys()), len(text_test1.keys()), len(sent_test1.keys()))  # 3531
# print(len(IDs_test2.keys()), len(text_test2.keys()), len(sent_test2.keys()))  # 1853
# print(len(IDs_test3.keys()), len(text_test3.keys()), len(sent_test3.keys()))  # 2379
# print(len(IDs_dev.keys()), len(text_dev.keys()), len(sent_dev.keys()))        # 2000


    # write code to read in the datasets here


335104872099066692
796528524030124618
760964834217238632
147713180324524046
732302280474120023
724826707518822134
134322773956255784
073060048261791634
855586456869811926
225466624830593540
033620536146652095
339455496639871586
254316794666070465
230746282258510561
145694051800713408
402401097268027978
538613241671440409
053238559290312048
904099478363202356
887813491640106727
224192301158773047
401798764118132130
820520494461810207
898915274896463860
752552454609796023
900003240536967046
164933585323466561
449359298613241597
449959629960520558
923245382385329070
465430385392606010
808422113535209971
608687728663961484
615225469270695927
002567481915962312
018628743318226386
365828687861220113
008285627009092502
046046618236971113
711214738643447655
875282374928102821
990537163660556822
245779628375149443
667949914431692444
804636191120565130
187860501654063649
747349010889760091
020318100288114779
220560002362050829
809954474384310925
904243490809984426
512315363232017856
975788842077

#### Build sentiment classifiers
You need to create your own classifiers (at least 3 classifiers). For each classifier, you can choose between the bag-of-word features and the word-embedding-based features. Each classifier has to be evaluated over 3 test sets. Make sure your classifier produce consistent performance across the test sets. Marking will be based on the performance over all 5 test sets (2 of them are not provided to you).

In [ ]:
# Buid traditional sentiment classifiers. An example classifier name 'svm' is given
# in the code below. You should replace the other two classifier names
# with your own choices. For features used for classifier training, 
# the 'bow' feature is given in the code. But you could also explore the 
# use of other features.
for classifier in ['svm', '<classifier-2-name>', '<classifier-3-name>']:
    for features in ['bow', '<feature-2-name>']:
        # Skeleton: Creation and training of the classifiers
        if classifier == 'svm':
            # write the svm classifier here
            print('Training ' + classifier)
        elif classifier == '<classifier-2-name>':
            # write the classifier 2 here
            print('Training ' + classifier)
        elif classifier == '<classifier-3-name>':
            # write the classifier 3 here
            print('Training ' + classifier)
        elif classifier == 'LSTM':
            # write the LSTM classifier here
            if features == 'bow':
                continue
            print('Training ' + classifier)
        else:
            print('Unknown classifier name' + classifier)
            continue

        # Predition performance of the classifiers
        for testset in testsets:
            id_preds = {}
            # write the prediction and evaluation code here

            testset_name = testset
            testset_path = join('semeval-tweets', testset_name)
            evaluate(id_preds, testset_path, features + '-' + classifier)